<a href="https://colab.research.google.com/github/2403a54123-web/NLP/blob/main/2403A54123_NLP_Lab_Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import string
from collections import Counter, defaultdict
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import math

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
text = """
Natural language processing is a field of artificial intelligence.
It focuses on interaction between computers and humans.
Language models help machines understand text.
N-gram models are simple but useful language models.
They predict words based on previous context.
"""

print(text[:300])



Natural language processing is a field of artificial intelligence.
It focuses on interaction between computers and humans.
Language models help machines understand text.
N-gram models are simple but useful language models.
They predict words based on previous context.



In [4]:
nltk.download('punkt_tab')
def preprocess_text(text):
    text = text.lower()  # lowercase

    text = re.sub(r'[0-9]', '', text)  # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))

    sentences = sent_tokenize(text)

    stop_words = set(stopwords.words('english'))

    processed_sentences = []

    for sent in sentences:
        words = word_tokenize(sent)
        words = [w for w in words if w not in stop_words]

        words = ['<s>'] + words + ['</s>']  # start/end tokens
        processed_sentences.append(words)

    return processed_sentences

processed_data = preprocess_text(text)
processed_data[:2]


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[['<s>',
  'natural',
  'language',
  'processing',
  'field',
  'artificial',
  'intelligence',
  'focuses',
  'interaction',
  'computers',
  'humans',
  'language',
  'models',
  'help',
  'machines',
  'understand',
  'text',
  'ngram',
  'models',
  'simple',
  'useful',
  'language',
  'models',
  'predict',
  'words',
  'based',
  'previous',
  'context',
  '</s>']]

In [5]:
def build_ngrams(data, n):
    ngrams = []
    for sentence in data:
        for i in range(len(sentence)-n+1):
            ngrams.append(tuple(sentence[i:i+n]))
    return Counter(ngrams)

unigram = build_ngrams(processed_data, 1)
bigram = build_ngrams(processed_data, 2)
trigram = build_ngrams(processed_data, 3)

print("Unigram sample:", list(unigram.items())[:5])
print("Bigram sample:", list(bigram.items())[:5])
print("Trigram sample:", list(trigram.items())[:5])


Unigram sample: [(('<s>',), 1), (('natural',), 1), (('language',), 3), (('processing',), 1), (('field',), 1)]
Bigram sample: [(('<s>', 'natural'), 1), (('natural', 'language'), 1), (('language', 'processing'), 1), (('processing', 'field'), 1), (('field', 'artificial'), 1)]
Trigram sample: [(('<s>', 'natural', 'language'), 1), (('natural', 'language', 'processing'), 1), (('language', 'processing', 'field'), 1), (('processing', 'field', 'artificial'), 1), (('field', 'artificial', 'intelligence'), 1)]


In [6]:
def bigram_prob(w1, w2):
    return bigram[(w1, w2)] / unigram[(w1,)]


In [7]:
V = len(unigram)

def laplace_bigram_prob(w1, w2):
    return (bigram[(w1, w2)] + 1) / (unigram[(w1,)] + V)


In [8]:
sentences = [
    "language models predict words",
    "natural language processing field",
    "computers understand language",
    "artificial intelligence models",
    "text prediction useful"
]


In [9]:
def sentence_bigram_prob(sentence):
    words = ['<s>'] + sentence.lower().split() + ['</s>']
    prob = 1

    for i in range(len(words)-1):
        prob *= laplace_bigram_prob(words[i], words[i+1])

    return prob

for s in sentences:
    print(s, sentence_bigram_prob(s))


language models predict words 8.708535758408963e-07
natural language processing field 1.2504564165920563e-06
computers understand language 2.0319916769620914e-06
artificial intelligence models 4.063983353924183e-06
text prediction useful 2.2758306781975425e-06


In [10]:
def perplexity(sentence):
    words = ['<s>'] + sentence.lower().split() + ['</s>']
    N = len(words)
    prob = sentence_bigram_prob(sentence)

    return pow(1/prob, 1/N)

for s in sentences:
    print(s, perplexity(s))


language models predict words 10.233145364207008
natural language processing field 9.634338626235857
computers understand language 13.75357547490397
artificial intelligence models 11.973182877013409
text prediction useful 13.44534685946749
